## Das neue Flatfile-CSV-Format (ffcsv) der Datenbank GENESIS-Online

https://www-genesis.destatis.de/datenbank/beta/announcement  
  
Lauffähiger Python/Pandas Code zum Einlesen

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 60

#### Das bleibt konstant:  
* Datensatz-orientiertes Format zur automatisierten Weiterverarbeitung
* Spalten durch Semikolon separiert (de, en)
* Dezimalkomma je nach Sprache "," (de) oder "." (en)
* utf-8 Codierung
* Zellen können wertersetzende Qualitätskennzeichen enthalten wie "-", "x", ".", "/"
* Spalte(n) Qualitätskennzeichen mit "_q" Suffix beim Download an- oder abwählbar

## Beispiel 1
`Tabelle: 61111-0001`  
**Verbraucherpreisindex: Deutschland, Jahre**  

#### Das bisherige ffcsv-Format  
* Preisindex und Veränderungsrate in getrennten Spalten
* Statistikspezifische Spaltenüberschriften

In [2]:
tabelle61111_0001alt = pd.read_csv("61111-0001_de_flat.csv", sep=";", decimal=",", 
                                   na_values=["-", "x", ".", "/"])

In [3]:
# ausgewählte Spalten
tabelle61111_0001alt[["Zeit","PREIS1__Verbraucherpreisindex__2020=100","PREIS1__Verbraucherpreisindex__q",
                   "Verbraucherpreisindex__CH0004","Verbraucherpreisindex__CH0004__q"]].tail(3)

,Zeit,PREIS1__Verbraucherpreisindex__2020=100,PREIS1__Verbraucherpreisindex__q,Verbraucherpreisindex__CH0004,Verbraucherpreisindex__CH0004__q
30,2021,103.1,e,3.1,e
31,2022,110.2,e,6.9,e
32,2023,116.7,e,5.9,e


#### Das neue ffcsv-Format  
* Einheitliche englische Spaltenbeschriftungen für de/en 
* Eine Wertespalte "value" unabhängig von der Statistik
* Eine Spalte "value_unit" für die Einheit
* Auslieferung als ZIP-Datei (eine ffcsv-Datei je ZIP-Archiv)
* unsortiert

In [4]:
tabelle61111_0001neu = pd.read_csv("61111-0001_de_flat.zip", sep=";", 
                                   decimal=",", na_values=["-", "x", ".", "/"])\
                                .sort_values(by=["time","value_variable_label"])

In [5]:
# ausgewählte Spalten
tabelle61111_0001neu[["time","value","value_unit",
                      "value_variable_code","value_variable_label","value_q"]].tail(6)

,time,value,value_unit,value_variable_code,value_variable_label,value_q
47,2021,103.1,2020=100,PREIS1,Verbraucherpreisindex,e
46,2021,3.1,%,PREIS1,in,e
45,2022,110.2,2020=100,PREIS1,Verbraucherpreisindex,e
44,2022,6.9,%,PREIS1,in,e
41,2023,116.7,2020=100,PREIS1,Verbraucherpreisindex,e
40,2023,5.9,%,PREIS1,in,e


**Tabellierung der Datensätze**

In [6]:
tabelle61111_0001neu.pivot(index="time", columns="value_unit", values="value").tail(3)

value_unit,%,2020=100
time,,
2021,3.1,103.1
2022,6.9,110.2
2023,5.9,116.7


## Beispiel 2
`Tabelle: 61111-0003`  
**Verbraucherpreisindex: Deutschland, Jahre, Klassifikation der Verwendungszwecke des Individualkonsums (COICOP 2-5-Steller Hierarchie)**  

#### Das bisherige ffcsv-Format  
* COICOP 2- bis 5-Steller gemischt in einer Spalte "2_Auspraegung_Code"
* 2- bis 5-Steller mussten über die Länge der Ausprägungscodes gefiltert werden

In [7]:
tabelle61111_0003alt = pd.read_csv("61111-0003_de_flat.csv", sep=";", decimal=",", 
                                   na_values=["-", "x", ".", "/"])

In [8]:
# ausgewählte Spalten
tabelle61111_0003alt[["Zeit","2_Merkmal_Code","2_Merkmal_Label",
                      "2_Auspraegung_Code", "2_Auspraegung_Label",
                      "PREIS1__Verbraucherpreisindex__2020=100"]].tail(15)

,Zeit,2_Merkmal_Code,2_Merkmal_Label,2_Auspraegung_Code,2_Auspraegung_Label,PREIS1__Verbraucherpreisindex__2020=100
1910,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-1253,Versicherungsdienstleistungen für die Gesundheit,107.0
1911,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12532,Dienstl. priv. Unfall- und Krankenversicherungen,107.0
1912,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-1254,Versicherungsdienstleistungen für den Verkehr,110.6
1913,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12541,Versicherungsdienstl. für private Verkehrsmittel,110.6
1914,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12542,Versicherungsdienstleistungen für Reise und Gepäck,108.8
1915,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-1255,Andere Versicherungsdienstleistungen,106.4
1916,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12550,Andere Versicherungsdienstleistungen,106.4
1917,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-1262,"Andere Finanzdienstleistungen, a.n.g.",109.8
1918,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12621,Bank- und Sparkassengebühren und Ähnliches,109.8
1919,2023,CC13A5,Verwendungszwecke des Individualkonsums,CC13-12622,"Gebühren für Anlageberatung, Steuerberatung u.Ä.",109.9


**Für das Berichtsjahr 2023 enthält die bisherige ffcsv-Datei**
* 385 Ausprägungen zum Merkmal "Verwendungszwecke des Individualkonsums" (Merkmalscode = CC13A5)

In [9]:
tabelle61111_0003alt.query("Zeit==2023")["2_Merkmal_Code"].value_counts()

2_Merkmal_Code
CC13A5    385
Name: count, dtype: int64

#### Das neue ffcsv-Format
* Über die Spalte "2_variable_code" werden die verschiedenen COICOP-Hierarchien filterbar
* Jede COICOP-Hierarchiestufe ist separat vollständig auswertbar

In [10]:
tabelle61111_0003neu = pd.read_csv("61111-0003_de_flat.zip", sep=";", 
                                   decimal=",", na_values=["-", "x", ".", "/"])

In [11]:
tabelle61111_0003neu["2_variable_label"] = tabelle61111_0003neu["2_variable_label"].str.replace("-","‑") # non-breaking hyphen
tabelle61111_0003neu[tabelle61111_0003neu["2_variable_attribute_code"].str.contains("CC13-12")]\
                    [["time","2_variable_code", "2_variable_label",
                      "2_variable_attribute_code","2_variable_attribute_label","value","value_q"]]\
                    .sort_values(by=["time","2_variable_attribute_code"]).tail(15)                                                                     

,time,2_variable_code,2_variable_label,2_variable_attribute_code,2_variable_attribute_label,value,value_q
260,2023,CC13A4,"Verwendungszwecke des Individualkonsums, 4‑Steller",CC13-1254,Versicherungsdienstleistungen für den Verkehr,110.6,e
1383,2023,CC13A5,"Verwendungszwecke des Individualkonsums, 5‑Steller",CC13-12541,Versicherungsdienstl. für private Verkehrsmittel,110.6,e
1472,2023,CC13A5,"Verwendungszwecke des Individualkonsums, 5‑Steller",CC13-12542,Versicherungsdienstleistungen für Reise und Gepäck,108.8,e
328,2023,CC13A4,"Verwendungszwecke des Individualkonsums, 4‑Steller",CC13-1255,Andere Versicherungsdienstleistungen,106.4,e
1650,2023,CC13A5,"Verwendungszwecke des Individualkonsums, 5‑Steller",CC13-12550,Andere Versicherungsdienstleistungen,106.4,e
639,2023,CC13A3,"Verwendungszwecke des Individualkonsums, 3‑Steller",CC13-126,"Finanzdienstleistungen, a.n.g.",109.8,e
489,2023,CC13A4,"Verwendungszwecke des Individualkonsums, 4‑Steller",CC13-1262,"Andere Finanzdienstleistungen, a.n.g.",109.8,e
1686,2023,CC13A5,"Verwendungszwecke des Individualkonsums, 5‑Steller",CC13-12621,Bank- und Sparkassengebühren und Ähnliches,109.8,e
1658,2023,CC13A5,"Verwendungszwecke des Individualkonsums, 5‑Steller",CC13-12622,"Gebühren für Anlageberatung, Steuerberatung u.Ä.",109.9,e
745,2023,CC13A3,"Verwendungszwecke des Individualkonsums, 3‑Steller",CC13-127,"Andere Dienstleistungen, a.n.g.",113.2,e


**Für das Berichtsjahr 2023 enthält die neue ffcsv-Datei**
* 275 Ausprägungen zum Merkmal "Verwendungszwecke des Individualkonsums, 5-Steller" (Merkmalscode = CC13A5)
* 110 Ausprägungen zum Merkmal "Verwendungszwecke des Individualkonsums, 4-Steller" (Merkmalscode = CC13A4)
* 44 Ausprägungen zum Merkmal "Verwendungszwecke des Individualkonsums, 3-Steller" (Merkmalscode = CC13A3)
* 12 Ausprägungen zum Merkmal "Verwendungszwecke des Individualkonsums, 2-Steller" (Merkmalscode = CC13A2)

In [12]:
tabelle61111_0003neu.query("time==2023")["2_variable_code"].value_counts()

2_variable_code
CC13A5    275
CC13A4    110
CC13A3     44
CC13A2     12
Name: count, dtype: int64

**Tabellierung der Datensätze**

In [13]:
tabelle61111_0003neu.pivot(index=["2_variable_label","2_variable_attribute_label"], 
                                    columns="time", values="value")

time                                                                                                   2019  \
2_variable_label                                   2_variable_attribute_label                                 
Verwendungszwecke des Individualkonsums, 2‑Steller Alkoholische Getränke und Tabakwaren                98.1   
                                                   Andere Waren und Dienstleistungen                   97.5   
                                                   Bekleidung und Schuhe                              101.7   
                                                   Bildungswesen                                      100.2   
                                                   Freizeit, Unterhaltung und Kultur                  100.3   
...                                                                                                     ...   
Verwendungszwecke des Individualkonsums, 5‑Steller Ärztliche Dienstleistungen der Allgemeinmedizin    100.0   
                                                   Ärztliche Dienstleistungen der Fachmedizin         100.0   
                                                   Übernachtungen in Feriendörfern, Camping u.Ä.       97.1   
                                                   Übernachtungen in Hotels, Gasthöfen und Ähnlichem   99.7   
                                                   Übernachtungen in anderen Unterkünften              98.3   

time                                                                                                   2020  \
2_variable_label                                   2_variable_attribute_label                                 
Verwendungszwecke des Individualkonsums, 2‑Steller Alkoholische Getränke und Tabakwaren               100.0   
                                                   Andere Waren und Dienstleistungen                  100.0   
                                                   Bekleidung und Schuhe                              100.0   
                                                   Bildungswesen                                      100.0   
                                                   Freizeit, Unterhaltung und Kultur                  100.0   
...                                                                                                     ...   
Verwendungszwecke des Individualkonsums, 5‑Steller Ärztliche Dienstleistungen der Allgemeinmedizin    100.0   
                                                   Ärztliche Dienstleistungen der Fachmedizin         100.0   
                                                   Übernachtungen in Feriendörfern, Camping u.Ä.      100.0   
                                                   Übernachtungen in Hotels, Gasthöfen und Ähnlichem  100.0   
                                                   Übernachtungen in anderen Unterkünften             100.0   

time                                                                                                   2021  \
2_variable_label                                   2_variable_attribute_label                                 
Verwendungszwecke des Individualkonsums, 2‑Steller Alkoholische Getränke und Tabakwaren               103.5   
                                                   Andere Waren und Dienstleistungen                  103.8   
                                                   Bekleidung und Schuhe                              101.5   
                                                   Bildungswesen                                      102.5   
                                                   Freizeit, Unterhaltung und Kultur                  102.9   
...                                                                                                     ...   
Verwendungszwecke des Individualkonsums, 5‑Steller Ärztliche Dienstleistungen der Allgemeinmedizin    100.0   
                                                   Ärztliche Dienstleistungen der Fachmedizin         100.0   
  

## Beispiel 3
`Tabelle: 21611-0002`  
**Kinos, Leinwände, Sitzplätze der Kinos, Filmbesuche, Durchschnittlicher Kino-Eintrittspreis, Einnahmen, Filmabgabe: Deutschland, Jahre**  

#### Das bisherige ffcsv-Format  
* Statistik- und sprachspezifische (de/en) Spaltenüberschriften, die sich aus Code, Label und Einheit zusammensetzen

In [14]:
tabelle21611_0002alt = pd.read_csv("21611-0002_de_flat.csv", sep=";", decimal=",", 
                                   na_values=["-", "x", ".", "/"])

In [15]:
# ausgewählte Spalten
tabelle21611_0002alt[["Zeit","FILM11__Kinos__Anzahl","FILM08__Sitzplaetze_der_Kinos__EUR",
                      "FILM05__Filmbesuche__Mill._EUR","FILM10__Filmbesuche_je_Einwohner__Mill._EUR"]].tail(2)

,Zeit,FILM11__Kinos__Anzahl,FILM08__Sitzplaetze_der_Kinos__EUR,FILM05__Filmbesuche__Mill._EUR,FILM10__Filmbesuche_je_Einwohner__Mill._EUR
21,2021,1723,8.87,373.2,369.2
22,2022,1730,9.26,722.0,715.0


#### Das neue ffcsv-Format  
* Getrennte Spalten für Code, Label und Einheit
* Programme statistikübergreifend nutzbar

In [16]:
tabelle21611_0002neu = pd.read_csv("21611-0002_de_flat.zip", sep=";", decimal=",", 
                                   na_values=["-", "x", ".", "/"])

In [17]:
# ausgewählte Spalten
tabelle21611_0002neu[["time","value","value_unit","value_variable_code",
                      "value_variable_label","value_q"]]\
                    .sort_values(by=["time","value_variable_code"]).tail(18)

,time,value,value_unit,value_variable_code,value_variable_label,value_q
126,2021,4931.00,Anzahl,FILM02,Leinwände,e
132,2021,789688.00,Anzahl,FILM03,Sitzplätze der Kinos,e
128,2021,42.10,Mill.,FILM04,Filmbesuche,e
127,2021,373.20,Mill. EUR,FILM05,Bruttoeinnahmen aus dem Filmbesuch,e
131,2021,0.50,Anzahl,FILM07,Filmbesuche je Einwohner,e
129,2021,8.87,EUR,FILM08,Durchschnittlicher Kino-Eintrittspreis,e
133,2021,4.00,Mill. EUR,FILM09,Filmabgabe der Kinos an die FFA,e
134,2021,369.20,Mill. EUR,FILM10,Nettoeinnahmen aus dem Filmbesuch (einschl. MwSt.),e
130,2021,1723.00,Anzahl,FILM11,Kinos,e
108,2022,4911.00,Anzahl,FILM02,Leinwände,e


**Tabellierung der Datensätze**

In [18]:
tabelle21611_0002neu.query("time>2020").pivot(index=["value_variable_label","value_unit"], 
                                              columns="time", values="value")

,time,2021,2022
value_variable_label,value_unit,,
Bruttoeinnahmen aus dem Filmbesuch,Mill. EUR,373.20,722.00
Durchschnittlicher Kino-Eintrittspreis,EUR,8.87,9.26
Filmabgabe der Kinos an die FFA,Mill. EUR,4.00,7.00
Filmbesuche,Mill.,42.10,78.00
Filmbesuche je Einwohner,Anzahl,0.50,0.90
Kinos,Anzahl,1723.00,1730.00
Leinwände,Anzahl,4931.00,4911.00
Nettoeinnahmen aus dem Filmbesuch (einschl. MwSt.),Mill. EUR,369.20,715.00
Sitzplätze der Kinos,Anzahl,789688.00,777844.00
